<a href="https://colab.research.google.com/github/phytometrics/plant_phenotyping_python/blob/dev/notebooks/leaf_venation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O data.zip https://zenodo.org/records/8020856/files/MorphometricsGroup/iwamasa-2022-v1.0.0.zip?download=1
!unzip data.zip
!rm data.zip

--2023-11-08 01:03:14--  https://zenodo.org/records/8020856/files/MorphometricsGroup/iwamasa-2022-v1.0.0.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.22.33, 188.185.33.206, 188.185.10.78, ...
Connecting to zenodo.org (zenodo.org)|188.185.22.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59997203 (57M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  57.22M  18.5MB/s    in 3.1s    

2023-11-08 01:03:17 (18.5 MB/s) - ‘data.zip’ saved [59997203/59997203]

Archive:  data.zip
a8f0e94cf10e8e5dc207b1c8127ff4aad16bad3b
   creating: MorphometricsGroup-iwamasa-2022-eea5c23/
  inflating: MorphometricsGroup-iwamasa-2022-eea5c23/.gitignore  
  inflating: MorphometricsGroup-iwamasa-2022-eea5c23/Dockerfile  
  inflating: MorphometricsGroup-iwamasa-2022-eea5c23/LICENSE  
  inflating: MorphometricsGroup-iwamasa-2022-eea5c23/README.md  
   creating: MorphometricsGroup-iwamasa-2022-eea5c23/data/
   creating: Mor

In [2]:
!pip install segmentation-models-pytorch==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 15.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12404 sha256=9fea45a929a690931e2b7a24293c6ce373562c5f5c50420470a7a3e65bb98fdb
  Stored in directory: /root/.cache/pip/wheels/61/3a/b0/0b4c443c380bd934701b0a25e4aed76479e4fcaf1a6f955664
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=36f84e8b4ca8d3a63e2b93d9d43508814d5cc40af235f7d49d5899a5e7f0b34d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


## 葉脈検出

In [3]:
import os
import shutil
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
import cv2
from tqdm.auto import tqdm
import joblib

import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as albu
import segmentation_models_pytorch as smp

# class Dataset(Dataset):
#     def __init__(
#             self,
#             data_dir=None,
#             augmentation=None,
#             preprocessing=None,):
#         self.image_paths = [p for p in glob(str(data_dir / '*.jpg'))]
#         self.mask_paths = [p.split('.jpg')[0] + '.npy' for p in self.image_paths]
#         self.augmentation = augmentation
#         self.preprocessing = preprocessing

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = cv2.imread(self.image_paths[idx])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         mask = np.load(self.mask_paths[idx])
#         mask = np.expand_dims(mask, axis=-1)

#         if self.augmentation:
#             sample = self.augmentation(image=image, mask=mask)
#             image, mask = sample['image'], sample['mask']

#         if self.preprocessing:
#             sample = self.preprocessing(image=image, mask=mask)
#             image, mask = sample['image'], sample['mask']

#         return image, mask

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing t      ransform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

def detect_green_color(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_min = np.array([30, 64, 0])
    hsv_max = np.array([90, 255, 255])
    mask = cv2.inRange(hsv, hsv_min, hsv_max)
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    return mask, masked_img

In [4]:
BASE_DIR = '/content/MorphometricsGroup-iwamasa-2022-eea5c23'
# INFERENCE_PATH = os.path.join(BASE_DIR, 'data/processed/segment-non-treated-dataset')
# IMAGE_DIR = os.path.join(BASE_DIR,  'data/interm/non-treated-dataset')
INFERENCE_PATH = '/content'

ENCODER = 'resnet18'
DECODER = 'unet'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'
crop_size = 512
buffer_size = 32
patch_size = crop_size - 2 * buffer_size

preprocessing_fn = smp.encoders.get_preprocessing_fn(
    ENCODER,
    ENCODER_WEIGHTS
)
preprocessing = get_preprocessing(preprocessing_fn)
best_model = torch.load(
    os.path.join(BASE_DIR , f'models/gray_{ENCODER}_{DECODER}.pth')
)

# image_paths = glob(os.path.join(IMAGE_DIR, '*'))
count = 0

# for i, path in tqdm(enumerate(image_paths), total=len(image_paths)):
path = "/content/MorphometricsGroup-iwamasa-2022-eea5c23/data/interm/non-treated-dataset/Zelkova_serrata_0.jpg"
file_name = "Zelkova_serrata_0.jpg"

# detect grean-area
im = cv2.imread(path)
contours, hierarchy = cv2.findContours(
    detect_green_color(im)[0],
    cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE
)
max_contours = max(contours, key=lambda x: cv2.contourArea(x))
mask = cv2.drawContours(
    np.zeros_like(im[:,:,0]),
    [max_contours],
    -1,
    color=255,
    thickness=-1
)
im = cv2.imread(path, 0)
im = np.where(mask==255, 255-im, 255)

im = np.expand_dims(im, axis=-1)
im = np.repeat(im, 3, axis=-1)
new_shape = [-(-im.shape[i]//patch_size)*patch_size + 2*buffer_size  for i in [0, 1]] + [3]
reshape_im = np.ones(new_shape, dtype=np.uint8) * 255
reshape_im[buffer_size:im.shape[0]+buffer_size, buffer_size:im.shape[1]+buffer_size, :] = im

output_im = np.zeros((new_shape[0], new_shape[1]), dtype=np.float32)
for h_i, h in enumerate(range(buffer_size, new_shape[0]-buffer_size, patch_size)):
    for w_i, w in enumerate(range(buffer_size, new_shape[1]-buffer_size, patch_size)):
        h -= buffer_size
        w -= buffer_size
        tmp_im = reshape_im[h:h+crop_size, w:w+crop_size, :]
        if reshape_im.mean() > 0:
            tmp_im = preprocessing(image=tmp_im)
            x_tensor = torch.from_numpy(tmp_im['image']).to(DEVICE).unsqueeze(0)
            tmp_output_im = best_model.predict(x_tensor).squeeze().cpu().numpy()
            h += buffer_size
            w += buffer_size
            output_im[h:h+patch_size, w:w+patch_size] = tmp_output_im[buffer_size:-buffer_size, buffer_size:-buffer_size]
output_im = ((output_im)*255).astype(np.uint8)
cv2.imwrite(os.path.join(INFERENCE_PATH, (file_name+'.png')), im)
cv2.imwrite(os.path.join(INFERENCE_PATH, ('vein_'+file_name+'.png')), output_im)

True

## skeletonization

In [5]:
import os
import shutil
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
import cv2
from tqdm.auto import tqdm
import joblib

from skimage.morphology import skeletonize


def get_skeletonizeimage(path, output_dir):
    new_path = path.split('/')[-1][5:][:-4]
    im = cv2.imread(path, 0)
    im = skeletonize(im>100).astype(np.uint8) * 255
    cv2.imwrite(os.path.join(output_dir, ('skeleton_'+new_path+'.png')), im)

get_skeletonizeimage("/content/vein_Zelkova_serrata_0.jpg.png", "./")

In [6]:
import os
import shutil
import random
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
import cv2
from tqdm.auto import tqdm
from collections import deque, defaultdict
import joblib


def get_nodes(im: np.array):
    H, W = im.shape
    im_nodes = np.zeros((H+2, W+2), dtype=int)
    for dh in [0, 1, 2]:
        for dw in [0, 1, 2]:
            if dh == 1 and dw == 1:
                im_nodes[dh:H+dh, dw:W+dw] += im * 100
            else:
                im_nodes[dh:H+dh, dw:W+dw] += im
    im_nodes = (im_nodes[1:H+1, 1:W+1]>=103)*255 + (im_nodes[1:H+1, 1:W+1]==101)*255

    return im_nodes


def get_nodepoints(im_nodes: np.array):
    H, W = im_nodes.shape
    nodes = defaultdict(list)
    _count = 0
    check = np.zeros_like(im_nodes, dtype=int)
    for h in range(H):
        for w in range(W):
            if check[h, w] == 1:
                continue
            check[h, w] = 1
            if im_nodes[h, w] == 255:
                nodes[_count] = [[h, w]]
                que = deque([[h, w]])
                while que:
                    _h, _w = que.popleft()
                    for dh in [-1, 0, 1]:
                        for dw in [-1, 0, 1]:
                            if dh == 0 and dw == 0:
                                continue
                            nh = _h + dh
                            nw = _w + dw
                            if 0<=nh<H and 0<=nw<W and check[nh, nw]==0 and im_nodes[nh, nw]==255:
                                nodes[_count].append([nh, nw])
                                check[nh, nw] = 1
                                que.append([nh, nw])
                _count += 1
    return nodes


def get_connectnodes(im: np.array, nodes: list):
    H, W = im.shape
    im_nodes_id = np.zeros_like(im, dtype=int) - 1
    for _key in nodes.keys():
        for h, w in nodes[_key]:
            im_nodes_id[h, w] = _key

    nodes_output = [[] for _ in range(len(nodes))]
    for _key in nodes.keys():
        que = deque(nodes[_key].copy())
        check = nodes[_key].copy()
        while que:
            _h, _w = que.popleft()
            for dh in [-1, 0, 1]:
                for dw in [-1, 0, 1]:
                    if dh == 0 and dw == 0: continue
                    nh = _h + dh
                    nw = _w + dw
                    if 0<nh<H and 0<=nw<W and im[nh, nw]==1 and (not [nh, nw] in check):
                        check.append([nh, nw])
                        if im_nodes_id[nh, nw] != -1:
                            nodes_output[_key].append(im_nodes_id[nh, nw])
                        else:
                            que.append([nh, nw])
    return nodes_output


def get_linkingnodes(connect_edges: defaultdict):
    nodes_check = [0] * len(connect_edges)
    nodes_size = [0] * len(connect_edges)

    for i in range(len(connect_edges)):
        if nodes_check[i] == 1: continue
        nodes_check[i] = 1

        que = deque([i])
        path_list = [i]
        while que:
            x = que.popleft()
            for y in connect_edges[x]:
                if nodes_check[y] == 1: continue
                nodes_check[y] = 1

                que.append(y)
                path_list.append(y)

        for _path in path_list:
            nodes_size[_path] = len(path_list)

    _max = max(nodes_size)
    _set = set([i for i in range(len(nodes_size)) if nodes_size[i] == _max])

    nodes_output = [[] for _ in range(len(connect_edges))]
    for x in range(len(connect_edges)):
        if not x in _set: continue
        for y in connect_edges[x]:
            if not y in _set: continue
            nodes_output[x].append(y)
    return nodes_output, _set


def get_network(path):
    new_path = path.split('/')[-1][9:][:-4]

    try:
        im = cv2.imread(path, 0)
        im = (im > 100).astype(int)
        im_nodes = get_nodes(im)
        nodes = get_nodepoints(im_nodes)
        connect_edges = get_connectnodes(im, nodes)
        connect_edges_linking, nodes_linking = get_linkingnodes(connect_edges)

        # out_path = new_path + '.txt'
        out_path = new_path + 'node-link.txt'
        with open(os.path.join(LINKOUTPUT_DIR, out_path), 'wb') as f:
            pickle.dump(connect_edges_linking, f)

        # out_path = new_path + '.json'
        out_path = new_path + 'node-position.json'
        with open(os.path.join(POSOUTPUT_DIR, out_path), 'wb') as f:
            pickle.dump(nodes, f)
        return None

    except:
        out_path

In [7]:
# BASE_DIR = '../'
# IMAGE_DIR = os.path.join(BASE_DIR, 'data/interm/skeleton-non-treated-dataset')
# OUTPUT_DIR = os.path.join(BASE_DIR, 'data/processed/network-non-treated-dataset')
# LINKOUTPUT_DIR = os.path.join(OUTPUT_DIR, 'node-link')
# POSOUTPUT_DIR = os.path.join(OUTPUT_DIR, 'node-position')

OUTPUT_DIR = "./"
LINKOUTPUT_DIR = "./"
POSOUTPUT_DIR = "./"

# image_paths = glob(os.path.join(IMAGE_DIR, '*'))

get_network("/content/skeleton_Zelkova_serrata_0.jpg.png")


## calculate network features

In [8]:
import os
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from glob import glob
from tqdm.auto import tqdm
import joblib

import re
import cv2
import pickle
from scipy.stats import skew, kurtosis
import networkx as nx
from networkx.generators.ego import ego_graph


from collections import deque, defaultdict


def get_netsimile(path):
    with open(path, "rb") as f:
        link = pickle.load(f)
    features = []
    G = nx.Graph()
    for u, e in enumerate(link):
        for v in e:
            G.add_edge(u, v)

    # degree
    ids = [i for i, p in enumerate(link) if p]
    degrees = {i: nx.degree(G)[i] for i in ids}

    # clustering_coefficient
    clustering = nx.clustering(G)

    # neighbor(one-hop)
    neighbor_degrees = {}
    for u, e in enumerate(link):
        if not e: continue
        d = []
        for v in e:
            d.append(degrees[v])
        neighbor_degrees[u] = sum(d) / len(d)

    # clustering_neighbor(one-hop)
    neighbor_clustering = {}
    for u, e in enumerate(link):
        if not e: continue
        d = []
        for v in e:
            d.append(clustering[v])
        neighbor_clustering[u] = sum(d) / len(d)

    ego_in_degree = []
    ego_out_degree = []
    ego_neighbor_nodes = []

    for u, e in enumerate(link):
        if not e: continue
        ego1 = ego_graph(G, u, radius=1)
        ego2 = ego_graph(G, u, radius=2)

        # edges_in_egonet(one-hop)
        ego_in_degree.append(ego1.number_of_edges())

        # edges_outgoing_from_egonet(one-hop)
        ego_out_degree.append(sum(dict(ego1.degree()).values()) - ego1.number_of_edges())

        # neighbor_of_egonet(one-hop)
        ego_neighbor_nodes.append(ego2.number_of_nodes() - ego1.number_of_nodes())


    output = pd.DataFrame({
        'degree': degrees.values(),
        'clustering_coefficient': clustering.values(),
        'neighbor(one-hop)': neighbor_degrees.values(),
        'clustering_neighbor(one-hop)': neighbor_clustering.values(),
        'edges_in_egonet(one-hop)': ego_in_degree,
        'edges_outgoing_from_egonet(one-hop)': ego_out_degree,
        'neighbor_of_egonet(one-hop)': ego_neighbor_nodes
    })

    for col in output.columns:
        values = output[col].values
        features.append(np.mean(values))
        features.append(np.std(values))
        features.append(skew(values))
        features.append(kurtosis(values))

    return [path, features]

In [9]:
columns = [
    'degree',
    'clustering_coefficient',
    'neighbor(one-hop)',
    'clustering_neighbor(one-hop)',
    'edges_in_egonet(one-hop)',
    'edges_outgoing_from_egonet(one-hop)',
    'neighbor_of_egonet(one-hop)'
]

statistics = [
    '_average',
    '_standard_deviation',
    '_skewness',
    '_kurtosis',
]

BASE_DIR = '../'
# NETWORK_DIR = os.path.join(BASE_DIR, 'data/processed/network-non-treated-dataset')
NETWORK_DIR = "./"
# link_paths = glob(os.path.join(NETWORK_DIR, 'node-link/*'))
link_paths = ["/content/Zelkova_serrata_0.jpgnode-link.txt"]
# 単一ファイル解析になおしておく

feature_cols = []
for col in columns:
    for sta in statistics:
        feature_cols.append(col + sta)

feature_results = joblib.Parallel(n_jobs=-1)(
    joblib.delayed(get_netsimile)(path) for path in tqdm(link_paths)
)

paths = [l[0].split('/')[-1][:-4] for l in feature_results]
features = [l[1] for l in feature_results]

feature_df = pd.concat([
    pd.DataFrame({'path': paths}),
    pd.DataFrame(features, columns=feature_cols)
], axis=1)
feature_df.to_csv(os.path.join(NETWORK_DIR,  'netsimile_features.csv'), index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
feature_df

,path,degree_average,degree_standard_deviation,degree_skewness,degree_kurtosis,clustering_coefficient_average,clustering_coefficient_standard_deviation,clustering_coefficient_skewness,clustering_coefficient_kurtosis,neighbor(one-hop)_average,...,edges_in_egonet(one-hop)_skewness,edges_in_egonet(one-hop)_kurtosis,edges_outgoing_from_egonet(one-hop)_average,edges_outgoing_from_egonet(one-hop)_standard_deviation,edges_outgoing_from_egonet(one-hop)_skewness,edges_outgoing_from_egonet(one-hop)_kurtosis,neighbor_of_egonet(one-hop)_average,neighbor_of_egonet(one-hop)_standard_deviation,neighbor_of_egonet(one-hop)_skewness,neighbor_of_egonet(one-hop)_kurtosis
0,Zelkova_serrata_0.jpgnode-link,2.508331,0.933899,-0.69121,-0.793665,0.007724,0.051841,7.72542,73.453735,2.889092,...,-0.521707,-0.563398,2.534139,0.968582,-0.521707,-0.563398,4.551672,1.954041,-0.062643,-1.127


## PCAとか